In [ ]:
program_url = 'http://www.wikicfp.com/cfp/program?id={}'
event_url = 'http://www.wikicfp.com/cfp/servlet/event.showcfp?eventid={}'
cfp_program_id = {
    'AAAI': 3, 
    'ACL': 37, 
    'CVPR': 628, 
    'ECCV': 800, 
    'EMNLP': 883, 
    'ICASSP': 1289, 
    'ICCV': 1325, 
    'ICIP': 1390, 
    'ICLR': -1, 
    'ICML': 1421, 
    'IJCAI': 1567, 
    'NIPS': 2212
}
cfp_event_id = {
    'TACL 2017': 59301
}

In [ ]:
from utils.Requests import Requests
from utils.helpers import parse_dates
from bs4 import BeautifulSoup as BS

def get_event_data(conf):
    id = cfp_event_id[conf]
    if id < 0: return {}
    
    r = Requests()
    text = r.get(event_url.format(id))
    soup = BS(text, 'html.parser')
    
    table = soup.find_all('table')[2].find_all('table')[3].find_all('table')[1]
    tokens = [td.text.strip() for td in table.find_all('td')]
    
    name = soup.find('title').text.strip()
    date = parse_dates(tokens[0].split(' - '))
    loca = tokens[1]
    subm = parse_dates([tokens[4], tokens[2]])
    
    result[conf] = {
        'name': name, 
        'date': date, 
        'location': loca, 
        'submission': subm
    }
    return result

def get_program_data(conf):
    id = cfp_program_id[conf]
    if id < 0: return {}
    
    r = Requests(verbose=True)
    text = r.get(program_url.format(id))
    soup = BS(text, 'html.parser')
    
    data = []
    for tr in soup.find_all('table')[2].find_all('table')[2].find_all('tr'):
        texts = [td.text for td in tr.find_all('td', {'align': 'left'})]
        a = tr.find('a')
        if a is not None:
            texts += [a['href']]
        if texts != []:
            data.append(texts)
    print(len(data))
    
    result = {}
    for i in range(0, len(data), 2):
        conf = data[i][0]
        name = data[i][1]
        event_url = data[i][2]
        date = parse_dates(data[i+1][0].split(' - '))
        loca = data[i+1][1]
        subm = parse_dates(data[i+1][2].strip('()').split(' ('))
        
        text = r.get(event_url)
        soup = BS(text, 'html.parser')
        url = soup.find('center').find_all('td', {'align': 'center'})[4].find('a')['href']
        
        result[conf] = {
            'name': name, 
            'date': date, 
            'location': loca, 
            'submission': subm, 
            'url': url
        }
    return result

# get_data('AAAI')

In [ ]:
result = {}
for conf, id in cfp_program_id.items():
    result.update(get_program_data(conf))
for conf, id in cfp_event_id.items():
    result.update(get_event_data(conf))
# result

In [ ]:
result.keys()

In [ ]:
result['CVPR 2017']

In [ ]:
import json
with open('conferences.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(result))